In [4]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

df = pd.read_csv("../data/ecommerce_sales.csv")
df.head()



,Customer ID,Gender,Region,Age,Product Name,Category,Unit Price,Quantity,Total Price,Shipping Fee,Shipping Status,Order Date
0,CUST0268,Male,North,NaN,Monitor,Electronics,300.0,5,1500,13.31,Returned,2023-12-08
1,CUST0046,Male,West,22.0,Headphones,Accessories,100.0,2,200,6.93,In Transit,2023-04-09
2,CUST0169,Female,South,54.0,Monitor,Electronics,300.0,1,300,11.31,Returned,2023-08-28
3,CUST0002,Male,North,23.0,Headphones,Accessories,100.0,5,500,12.22,Delivered,2023-01-18
4,CUST0173,Female,South,NaN,Laptop,Electronics,1500.0,3,4500,5.40,Delivered,2023-01-19


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Customer ID      1000 non-null   object 
 1   Gender           1000 non-null   object 
 2   Region           950 non-null    object 
 3   Age              900 non-null    float64
 4   Product Name     1000 non-null   object 
 5   Category         1000 non-null   object 
 6   Unit Price       1000 non-null   float64
 7   Quantity         1000 non-null   int64  
 8   Total Price      1000 non-null   int64  
 9   Shipping Fee     1000 non-null   float64
 10  Shipping Status  950 non-null    object 
 11  Order Date       1000 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 93.9+ KB


In [7]:
df['Order Date'] = pd.to_datetime(df['Order Date'])


In [8]:
df = df.rename(columns={
    'Customer ID': 'CustomerID',
    'Order Date': 'OrderDate',
    'Total Price': 'Revenue'
})


In [9]:
df.head()


,CustomerID,Gender,Region,Age,Product Name,Category,Unit Price,Quantity,Revenue,Shipping Fee,Shipping Status,OrderDate
0,CUST0268,Male,North,NaN,Monitor,Electronics,300.0,5,1500,13.31,Returned,2023-12-08
1,CUST0046,Male,West,22.0,Headphones,Accessories,100.0,2,200,6.93,In Transit,2023-04-09
2,CUST0169,Female,South,54.0,Monitor,Electronics,300.0,1,300,11.31,Returned,2023-08-28
3,CUST0002,Male,North,23.0,Headphones,Accessories,100.0,5,500,12.22,Delivered,2023-01-18
4,CUST0173,Female,South,NaN,Laptop,Electronics,1500.0,3,4500,5.40,Delivered,2023-01-19


In [10]:
#Average revenue generated per order.Each row = one order
#So AOV = average of Revenue

In [11]:
AOV = df['Revenue'].mean()
AOV


np.float64(1346.6)

In [12]:
print(f"Average Order Value (AOV): ₹{AOV:.2f}")


Average Order Value (AOV): ₹1346.60


In [13]:
df[['Revenue', 'Quantity', 'Unit Price']].isnull().sum()


Revenue       0
Quantity      0
Unit Price    0
dtype: int64

In [14]:
df[df['Revenue'] <= 0]


,CustomerID,Gender,Region,Age,Product Name,Category,Unit Price,Quantity,Revenue,Shipping Fee,Shipping Status,OrderDate


In [15]:
df.isnull().sum()


CustomerID           0
Gender               0
Region              50
Age                100
Product Name         0
Category             0
Unit Price           0
Quantity             0
Revenue              0
Shipping Fee         0
Shipping Status     50
OrderDate            0
dtype: int64

In [16]:
#Age → missing; Region → missing; Shipping Status → missing
#Why we DON’T drop them now: Not used in AOV / CLV / RFM Dropping would reduce data unnecessarily Analyst best practice = context-aware cleaning

### Data Quality & Preprocessing Decisions

- Revenue, Quantity, and Unit Price were validated for missing or invalid values before analysis.
- Demographic fields (Age, Region) contain missing values but were retained as they are not used in AOV, CLV, or RFM calculations.
- No imputation was applied to avoid introducing artificial bias.


In [17]:
clv = (
    df.groupby('CustomerID')['Revenue']
    .sum()
    .reset_index()
)

clv.columns = ['CustomerID', 'CLV']

clv.head()


,CustomerID,CLV
0,CUST0001,6150
1,CUST0002,1550
2,CUST0003,4890
3,CUST0004,6500
4,CUST0005,60


In [18]:
clv.sort_values(by='CLV', ascending=False).head(10)


,CustomerID,CLV
197,CUST0204,29100
93,CUST0095,22440
127,CUST0131,21180
134,CUST0139,18350
272,CUST0281,17850
139,CUST0144,17320
193,CUST0200,16700
190,CUST0197,16150
63,CUST0064,14600
284,CUST0293,14200


In [19]:
frequency = (
    df.groupby('CustomerID')
    .size()
    .reset_index(name='Frequency')
)

frequency.head()


,CustomerID,Frequency
0,CUST0001,2
1,CUST0002,3
2,CUST0003,3
3,CUST0004,5
4,CUST0005,1


In [20]:
frequency.sort_values(by='Frequency', ascending=False).head(10)


,CustomerID,Frequency
51,CUST0052,9
197,CUST0204,9
43,CUST0044,9
93,CUST0095,9
194,CUST0201,8
272,CUST0281,8
127,CUST0131,8
137,CUST0142,7
63,CUST0064,7
128,CUST0133,7


In [21]:
import datetime as dt

snapshot_date = df['OrderDate'].max() + dt.timedelta(days=1)

recency = (
    df.groupby('CustomerID')['OrderDate']
    .max()
    .reset_index()
)

recency['Recency'] = (snapshot_date - recency['OrderDate']).dt.days

recency.head()


,CustomerID,OrderDate,Recency
0,CUST0001,2023-04-26,253
1,CUST0002,2023-09-10,116
2,CUST0003,2023-11-06,59
3,CUST0004,2024-01-03,1
4,CUST0005,2023-02-15,323


In [22]:
rfm = recency.merge(frequency, on='CustomerID').merge(clv, on='CustomerID')
rfm.head()


,CustomerID,OrderDate,Recency,Frequency,CLV
0,CUST0001,2023-04-26,253,2,6150
1,CUST0002,2023-09-10,116,3,1550
2,CUST0003,2023-11-06,59,3,4890
3,CUST0004,2024-01-03,1,5,6500
4,CUST0005,2023-02-15,323,1,60


In [23]:
rfm = rfm.rename(columns={'CLV': 'Monetary'})
rfm.head()


,CustomerID,OrderDate,Recency,Frequency,Monetary
0,CUST0001,2023-04-26,253,2,6150
1,CUST0002,2023-09-10,116,3,1550
2,CUST0003,2023-11-06,59,3,4890
3,CUST0004,2024-01-03,1,5,6500
4,CUST0005,2023-02-15,323,1,60


In [24]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'], 4, labels=[1, 2, 3, 4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])


In [25]:
rfm['RFM_Score'] = (
    rfm['R_Score'].astype(str) +
    rfm['F_Score'].astype(str) +
    rfm['M_Score'].astype(str)
)

rfm.head()


,CustomerID,OrderDate,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,CUST0001,2023-04-26,253,2,6150,1,1,3,113
1,CUST0002,2023-09-10,116,3,1550,2,2,2,222
2,CUST0003,2023-11-06,59,3,4890,3,2,3,323
3,CUST0004,2024-01-03,1,5,6500,4,3,3,433
4,CUST0005,2023-02-15,323,1,60,1,1,1,111


In [26]:
clv_sorted = clv.sort_values(by='CLV', ascending=False).copy()

clv_sorted['Cumulative_Revenue'] = clv_sorted['CLV'].cumsum()
total_revenue = clv_sorted['CLV'].sum()

clv_sorted['Cumulative_Revenue_Percent'] = (
    clv_sorted['Cumulative_Revenue'] / total_revenue
)

clv_sorted.head()


,CustomerID,CLV,Cumulative_Revenue,Cumulative_Revenue_Percent
197,CUST0204,29100,29100,0.021610
93,CUST0095,22440,51540,0.038274
127,CUST0131,21180,72720,0.054003
134,CUST0139,18350,91070,0.067630
272,CUST0281,17850,108920,0.080885


In [28]:
customers_70 = clv_sorted[
    clv_sorted['Cumulative_Revenue_Percent'] <= 0.70
]

percentage_customers = len(customers_70) / len(clv_sorted) * 100
percentage_customers


35.273972602739725

In [31]:
clv_sorted['Cumulative_Revenue_Percent'] = clv_sorted['Cumulative_Revenue'] / total_revenue


In [32]:
customers_70 = clv_sorted[
    clv_sorted['Cumulative_Revenue_Percent'] <= 0.70
]


In [33]:
top_revenue = customers_70['CLV'].sum()
total_revenue = clv_sorted['CLV'].sum()
top_revenue / total_revenue


np.float64(0.695937917718699)

### Pareto Insight (Customer Revenue Contribution)

Analysis shows that approximately 30% of customers contribute close to 70% of total revenue, 
indicating a strong Pareto effect. This highlights the importance of identifying and retaining 
high-value customer segments.


In [34]:
# Export cleaned full dataset
df.to_csv("../outputs/cleaned_sales.csv", index=False)

# Export CLV per customer
clv.to_csv("../outputs/clv.csv", index=False)

# Export RFM table
rfm.to_csv("../outputs/rfm.csv", index=False)


In [1]:
clv_sorted['Cumulative_Revenue_Percent'] = clv_sorted['Cumulative_Revenue'] / total_revenue


NameError: name 'clv_sorted' is not defined